In [43]:
import pandas as pd
import numpy as np
import holoviews as hv
import panel as pn
import networkx as nx
import requests
import json

### SDG Data API

In [14]:
def all_geoareas():
    data = requests.get("https://unstats.un.org/SDGAPI/v1/sdg/GeoArea/List")
    return pd.DataFrame(data.json())
    
def all_goals():
    data = requests.get("https://unstats.un.org/SDGAPI/v1/sdg/Goal/List?includechildren=false")
    return pd.DataFrame(data.json())

def all_targets():
    data = requests.get("https://unstats.un.org/SDGAPI/v1/sdg/Target/List?includechildren=true")
    return pd.DataFrame(data.json())

def all_indicators():
    data = requests.get("https://unstats.un.org/SDGAPI/v1/sdg/Indicator/List")
    return pd.DataFrame(data.json())

def all_series():
    data = requests.get("https://unstats.un.org/SDGAPI/v1/sdg/Series/List")
    return pd.DataFrame(data.json())

In [3]:
ecosystem_capitals = pd.DataFrame([
            'Natural Capital', 
            'Intellectual Capital', 
            'Human Capital', 
            'Manufactured Capital', 
            'Financial Capital', 
            'Social & Relationship Capial',
        ],columns=['Ecosystem Capitals'])

In [15]:
all_geoareas = all_geoareas()
all_goals = all_goals()
all_targets = all_targets()
all_indicators = all_indicators()
all_series = all_series()

### GeoTree

In [237]:
def traverse(G, v=None):
    for n in G:
        nodes_data.append({'name':n['geoAreaName'],'group':n['type'],'code':n['geoAreaCode']})
        if v:
            links_data.append({'value':1,'source':v['geoAreaCode'],'target':n['geoAreaCode']})
        if n['children']:
            traverse(n['children'], n)
            
data = requests.get("https://unstats.un.org/SDGAPI/v1/sdg/GeoArea/Tree")
geo_tree = data.json()

data = requests.get("https://unstats.un.org/SDGAPI/v1/sdg/Goal/List?includechildren=true")
sdg_data = pd.DataFrame(data.json())

In [240]:
links_data = []
nodes_data = []
traverse(geo_tree)

### Nodes - Geo - Eco Capitals - SDGs - Zone of Life

In [241]:
geo_links = pd.DataFrame(links_data)
geo_links['source'] = geo_links['source'].apply(lambda x: 'g'+str(x))
geo_links['target'] = geo_links['target'].apply(lambda x: 'g'+str(x))
geo_nodes = pd.DataFrame(nodes_data)
geo_nodes['code'] = geo_nodes['code'].apply(lambda x: 'g'+str(x))
geo_nodes['size'] = 3

In [242]:
ecosystem_capitals_nodes = pd.DataFrame()
ecosystem_capitals_nodes['name'] = ecosystem_capitals['Ecosystem Capitals']
ecosystem_capitals_nodes['group'] = 'Ecosystem Capitals'
ecosystem_capitals_nodes['code'] = ecosystem_capitals.index

In [243]:
sdg_nodes = pd.DataFrame()
sdg_nodes['name'] = sdg_data['title']
sdg_nodes['group'] = 'SDG'
sdg_nodes['code'] = sdg_data['code']
sdg_nodes['size'] = 5

In [244]:
zone_of_life_node = pd.DataFrame([['Zone of Life','ZOL',9001, 20]],columns=['name','group','code','size'])

In [245]:
zone_of_life_node['code'] = zone_of_life_node['code'].apply(lambda x: 'z'+str(x))
sdg_nodes['code'] = sdg_nodes['code'].apply(lambda x: 's'+str(x))
ecosystem_capitals_nodes['code'] = ecosystem_capitals_nodes['code'].apply(lambda x: 'e'+str(x))

### Links - Geo - Eco Capitals - Zone of Life

In [246]:
eco_links1 = pd.DataFrame()
eco_links1['source'] = ecosystem_capitals_nodes['code']
eco_links1['target'] = 'g1'
eco_links1['value'] = '0.75'
eco_links2 = pd.DataFrame()
eco_links2['source'] = ecosystem_capitals_nodes['code']
eco_links2['target'] = 'z9001'
eco_links2['value'] = '0.75'
eco_links = pd.concat([eco_links1,eco_links2],sort=False)
                                
sdg_links = pd.DataFrame()
sdg_links['target'] = sdg_nodes['code']
sdg_links['source'] = 'z9001'
sdg_links['value'] = '0.25'

In [247]:
nodes_df = pd.concat([geo_nodes,ecosystem_capitals_nodes,sdg_nodes,zone_of_life_node],sort=False).reset_index()
links_df = pd.concat([geo_links,sdg_links,eco_links,],sort=False)
links_df.loc[len(links_df)] = [0.1,'g1','z9001']

### Map node codes

In [248]:
code_to_index = dict(zip(nodes_df['code'],nodes_df.index))
links_df_mapped = pd.DataFrame()
links_df_mapped['source'] = links_df['source'].map(code_to_index)
links_df_mapped['target'] = links_df['target'].map(code_to_index)

### Convert to JSON and dump to disk

In [251]:
links_list = links_df_mapped.to_dict('records')

nodes_list = nodes_df[['name','group']].to_dict('records')

json_prep = {"nodes":nodes_list, "links":links_list}

json_dump = json.dumps(json_prep, indent=1, sort_keys=True)

filename_out = 'pcap_export.json'
json_out = open(filename_out,'w')
json_out.write(json_dump)
json_out.close()

### Mapping Targets to SDGs

In [ ]:
target_nodes = pd.DataFrame()
target_nodes['name'] = all_targets['title']
target_nodes['group'] = 'Targets'
target_nodes['code'] = all_targets['code'].apply(lambda x: 'g'+str(x))
target_nodes['goal_code'] = all_targets['goal'].apply(lambda x: 'g'+str(x))
target_nodes['size'] = 3

target_links = pd.DataFrame()
target_links['source'] = target_nodes['code']
target_links['target'] = target_nodes['goal_code']
target_links['value'] = 1